In [25]:
%load_ext autoreload
%autoreload 2
import pyspark
import lsde2021.download as dl
import lsde2021.aggregate as agg
from lsde2021.types import PathLike
from dateutil.relativedelta import relativedelta
from pyspark.sql import SparkSession
from functools import partial
from pprint import pprint
from pathlib import Path
from typing import Tuple
import itertools
import pandas as pd
import datetime
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
MAX_MEMORY = "60G"

spark = SparkSession \
    .builder \
    .appName("data-statistics") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .config('spark.driver.maxResultSize', MAX_MEMORY) \
    .config('spark.ui.showConsoleProgress', 'false') \
    .getOrCreate()
sc = spark.sparkContext

21/10/10 22:47:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/10 22:47:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/10/10 22:47:14 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/10/10 22:47:14 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
21/10/10 22:47:14 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [26]:
# data exploration statistics
years = [2018, 2019, 2020, 2021]
months = range(1,13)
days = [1, 15]
pageview_samples = [
    datetime.datetime(year, month, day) for year, month, day in itertools.product(years, months, days)
]

pageview_complete_dir = Path("../hdd/pageview_complete")
pageview_sample_files = [
    pageview_complete_dir / "/".join(dl.wikimedia_pageview_complete_local_file(date, monthly=False))
    for date in pageview_samples
]
# pprint(pageview_sample_files)

In [52]:
print("checked %d of %d" % (
    len(list(pageview_complete_dir.rglob("**/*.ok"))), len(list(pageview_complete_dir.rglob("**/*.bz2")))
))

checked 182 of 1414


In [51]:
%%time
pageview_columns = ["wiki_code", "page_title", "page_id", "user_client", "daily_total", "hourly_count"]
pageview_dtype = {
    "wiki_code": "string",
    "page_title": "string",
    "page_id": "string", # will be parsed to int
    "user_client": "string",
    "daily_total": "string", # will be parsed to int
    "hourly_count": "string", # from 0 to 23, written as 0 = A, 1 = B ... 22 = W, 23 = X, will be parsed
}
df = pd.read_csv(
    "../hdd/pageview_complete/2018/2018-01/pageviews-20180101-user.csv",
    sep=' ',
    names=pageview_columns,
    low_memory=False,
    dtype=pageview_dtype,
)
df.head()

CPU times: user 17 s, sys: 1.25 s, total: 18.3 s
Wall time: 18.3 s


,wiki_code,page_title,page_id,user_client,daily_total,hourly_count
0,aa.wikibooks,Main_Page,desktop,10,F1I1K1L1N1S1U1V1W2,<NA>
1,aa.wikibooks,Main_Page,mobile-web,2,B1O1,<NA>
2,aa.wikibooks,Special:CreateAccount,desktop,3,F1N1W1,<NA>
3,aa.wikibooks,Special:Version,desktop,1,X1,<NA>
4,aa.wikibooks,User:EVula/header,desktop,1,U1,<NA>


In [ ]:
%%time
# load and combine all raw daily csv files
dfs = None
for date, f in tqdm(list(zip(pageview_samples, pageview_sample_files))[:2]):
    df = pd.read_csv(f, sep=' ', names=pageview_columns, low_memory=False, dtype=pageview_dtype)
    df["date"] = date
    if dfs is None:
        dfs = df
    else:
        dfs = pd.concat([dfs, df])
dfs.head()

  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
# % of pages with less than X views per day (can we remove them?)

In [9]:
# How many % of total page views make up the top N most popular pages?

In [11]:
# mean and std dev of page views per day (absolute)

# mean and std page views per country code per day (absolute and relative)

In [12]:
# mean and std pages visited per day (absolute)

# mean and std pages visited per country code per day (absolute and relative)

In [13]:
# for all pages, in how many other days do they occur? (relative)

# for all days, how many percent of pages are found in more than 90% of days (absolute and relative) 

In [14]:
# mean storage size per raw csv (daily and monthly)

In [15]:
# mean storage size per daily parquet file